<a href="https://colab.research.google.com/github/pablo-arantes/making-it-rain/blob/main/Glycam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hello there!**

This is a Jupyter notebook for running Molecular Dynamics (MD) simulations using OpenMM engine and AMBER/GLYCAM force field for **Protein** and **Glycosylated** systems. This notebook is a supplementary material of the paper "***Making it rain: Cloud-based molecular simulations for everyone***" ([link here](https://doi.org/10.1021/acs.jcim.1c00998)) and we encourage you to read it before using this pipeline.

The main goal of this notebook is to demonstrate how to harness the power of cloud-computing to run microsecond-long MD simulations in a cheap and yet feasible fashion.

---

 **This notebook is NOT a standard protocol for MD simulations!** It is just a simple MD pipeline illustrating each step of a simulation protocol.

---
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/pablo-arantes/making-it-rain/issues

**Acknowledgments**
- We would like to thank the OpenMM team for developing an excellent and open source engine.

- Thank you to [GLYCAM-web](https://glycam.org/#) server team, Professor Robert J. Woods, for their amazing web-tool and force field.

- Making-it-rain by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)).


- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.

- For related notebooks see: [Making-it-rain](https://github.com/pablo-arantes/making-it-rain)

# **Introduction**

In general, MD simulations rely on 1) a set of atomic coordinates of all atoms on a simulation box and 2) a set of force field parameters that describes the interaction energies between atoms.

In terms of inputs, we wil need:
*  A PDB ID or a .pdb file containing a set of atomic coordinates of your glycosylated system from [GLYCAM](https://glycam.org/#) server.

In this notebook, we will simulate a glycosylated protein, Ferrulate esterase (PDB ID 2HL6). To build our simulation box, we will use LEaP program (https://ambermd.org/tutorials/pengfei/index.php). The LEaP program is a portal between many chemical structure file types (.pdb and .mol2, primarily), and the Amber model parameter file types such as .lib, .prepi, parm.dat, and .frcmod. Each of the parameter files contains pieces of information needed for constructing a simulation, whether for energy minimization or molecular dynamics. LEaP functions within a larger workflow described in Section 1.1 of the [Amber Manual](https://ambermd.org/doc12/Amber20.pdf);.

You can download the input files examples from [here](https://github.com/pablo-arantes/making-it-rain/tree/main/AMBER_GLYCAM);
## ---







---
---
# **Setting the environment for MD calculation**

Firstly, we need to install all necessary libraries and packages for our simulation. The main packages we will be installing are:

1.    Anaconda (https://docs.conda.io/en/latest/miniconda.html)
2.    OpenMM (https://openmm.org/)
3.    PyTraj (https://amber-md.github.io/pytraj/latest/index.html)
4.    py3Dmol (https://pypi.org/project/py3Dmol/)
5.    Numpy (https://numpy.org/)
6.    Matplotlib (https://matplotlib.org/)
7.    AmberTools (https://ambermd.org/AmberTools.php)

In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
# install dependencies
%%capture
!pip -q install py3Dmol
!pip install git+https://github.com/pablo-arantes/biopandas
!mamba install openmm=7.7.0
!mamba install -c conda-forge ambertools -y
!pip install --upgrade MDAnalysis==2.4.2

#load dependencies
import sys
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats

## Using Google Drive to store simulation data

Google Colab does not allow users to keep data on their computing nodes. However, we can use Google Drive to read, write, and store our simulations files. Therefore, we suggest to you to:

1.   Create a folder in your own Google Drive and copy the necessary input files there.
2.   Copy the path of your created directory. We will use it below.

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Check if you correctly allocated GPU nodes**

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

---
---
# **Loading the necessary input files**

At this point, we should have all libraries and dependencies installed and all necessary input files already at your Google Drive folder.

**Important**: Make sure the PDB file points to the correct pathway. If necessary, correct the pathway and re-upload the files.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

**Please, don't use spaces in the files and folders names, use _ instead i.e., protein_glycan.pdb, MyDrive/protein_glycan and so on.**

In [ ]:
#@title **Please, provide the necessary input files below**:
#@markdown **Important:** This notebook uses as input, the pdb format from AMBER/GLYCAM package. You **MUST** use the pdb from GLYCAM server (https://glycam.org/). Probably you will have an error in topology generation, if you use a different PDB format.
PDB_file_name = 'structure.pdb' #@param {type:"string"}
file_name = PDB_file_name
Google_Drive_Path = '/content/drive/MyDrive/protein_glycan' #@param {type:"string"}
workDir = Google_Drive_Path
initial_pdb = os.path.join(workDir, str(file_name))
# starting = os.path.join(workDir, "starting.pdb")
# starting2 = os.path.join(workDir, str(file_name))
starting_end = os.path.join(workDir, str(file_name))
tleap = os.path.join(workDir, "tleap.in")
top_nw = os.path.join(workDir, "SYS_nw.prmtop")
crd_nw = os.path.join(workDir, "SYS_nw.crd")
pdb_nw = os.path.join(workDir, "SYS_nw.pdb")
top = os.path.join(workDir, "SYS.prmtop")
crd = os.path.join(workDir, "SYS.crd")
pdb = os.path.join(workDir, "SYS.pdb")

# pdb4amber_cmd = "pdb4amber -i " + str(starting2) + " -o " + str(starting_end) + " -p"

# original_stdout = sys.stdout # Save a reference to the original standard output

# with open('pdb4amber.sh', 'w') as f:
#     sys.stdout = f # Change the standard output to the file we created.
#     print(pdb4amber_cmd)
#     sys.stdout = original_stdout # Reset the standard output to its original value

# !chmod 700 pdb4amber.sh 2>&1 1>/dev/null
# !bash pdb4amber.sh

#@markdown ---

In [ ]:
#@title **Parameters to generate the Amber topology:**

Force_field = "ff19SB" #@param ["ff19SB", "ff14SB"]
if Force_field == "ff19SB":
  ff = "leaprc.protein.ff19SB"
else:
  ff = "leaprc.protein.ff14SB"

Water_type = "TIP3P" #@param ["TIP3P", "OPC"]
if Water_type == "TIP3P":
  water = "leaprc.water.tip3p"
  water_box = "TIP3PBOX"
else:
  water = "leaprc.water.opc"
  water_box = "OPCBOX"

#@markdown Size Box (Angstrons):

Size_box = 12 #@param {type:"slider", min:10, max:20, step:1}
size_box = Size_box

#@markdown **ATTENTION**: Give the concentration in Molar units, AMBER tleap will neutralize your system automatically:

Ions = "NaCl" #@param ["NaCl", "KCl" ]

Concentration = "0.15" #@param {type:"string"}

#@markdown ---

f = open(tleap, "w")
f.write("""source """ + str(ff) + "\n"
"""source leaprc.DNA.OL15
source leaprc.RNA.OL3
source leaprc.GLYCAM_06j-1
source leaprc.gaff2
source """  + str(water) + "\n"
"""SYS = loadpdb """  + str(starting_end) + "\n"
"""alignaxes SYS
savepdb SYS """ + str(pdb_nw) + "\n"
"""saveamberparm SYS """ + str(top_nw) + " " + str(crd_nw) + "\n"
"""solvatebox SYS """ + str(water_box) + " " + str(size_box) +  """ 0.7
saveamberparm SYS """ + str(top) + " " + str(crd) + "\n"
"""savepdb SYS """ + str(pdb) + "\n"
"""quit""")
f.close()

tleap_command = "tleap -f " + str(tleap)

original_stdout = sys.stdout # Save a reference to the original standard output

with open('run_tleap.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(tleap_command)
    sys.stdout = original_stdout # Reset the standard output to its original value

SYS = os.path.join(workDir, "SYS*")
rm_sys = "rm " + SYS

original_stdout = sys.stdout # Save a reference to the original standard output

with open('rm_sys.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(rm_sys)
    sys.stdout = original_stdout # Reset the standard output to its original value

!chmod 700 rm_sys.sh 2>&1 1>/dev/null
!bash rm_sys.sh 2> /dev/null

!chmod 700 run_tleap.sh 2>&1 1>/dev/null
!bash run_tleap.sh 2>&1 1>/dev/null

!grep "Volume:" leap.log > temp.txt
with open("temp.txt", 'r') as f:
  for line in f:
        vol = float(line.split()[1])

vol_lit  = vol * pow(10, -27)
atom_lit = 9.03 * pow(10, 22)
conc = float(Concentration)
num_ion = int(vol_lit * (conc/0.15) * atom_lit)

if Ions == "NaCl":
  pos_neut = "Na+ 0"
  pos_num = "Na+ " + str(num_ion)
  Cl_num = num_ion
else:
  pos_neut = "K+ 0"
  pos_num = "K+ " + str(num_ion)
  Cl_num = num_ion

f = open(tleap, "w")
f.write("""source """ + str(ff) + "\n"
"""source leaprc.DNA.OL15
source leaprc.RNA.OL3
source leaprc.GLYCAM_06j-1
source leaprc.gaff2
source """  + str(water) + "\n"
"""SYS = loadpdb """  + str(starting_end) + "\n"
"""alignaxes SYS
check SYS
charge SYS
addions SYS """ + str(pos_neut) + "\n"
"""addions SYS Cl- 0
check SYS
charge SYS
savepdb SYS """ + str(pdb_nw) + "\n"
"""saveamberparm SYS """ + str(top_nw) + " " + str(crd_nw) + "\n"
"""solvatebox SYS """ + str(water_box) + " " + str(size_box) +  """ 0.7 """ + "\n"
"""addIonsRand SYS """ + str(pos_num) + """ Cl- """ + str(Cl_num) + "\n"
"""saveamberparm SYS """ + str(top) + " " + str(crd) + "\n"
"""savepdb SYS """ + str(pdb) + "\n"
"""quit""")
f.close()

!chmod 700 run_tleap.sh 2>&1 1>/dev/null
# !bash run_tleap.sh 2>&1 1>/dev/null
!bash run_tleap.sh 2>&1 1>/dev/null


pdb_amber = os.path.exists(pdb)
top_amber = os.path.exists(top)
crd_amber = os.path.exists(crd)

if pdb_amber == True and top_amber == True and crd_amber == True:
  print("Successfully generated topology! :-)")
else:
  print("ERROR: Check if your input file is in AMBER/GLYCAM format! ")
!rm *.sh temp.txt >/dev/null 2>&1

## Let's take a look on our simulation box:

In [ ]:
#@title **Show 3D structure**
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

color = "gray" #@param ["gray", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_glycan = True #@param {type:"boolean"}
show_box = True #@param {type:"boolean"}
box_opacity = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}


def show_pdb(show_sidechains=False, show_mainchains=False, show_glycan=False, show_box = False, color="rainbow"):
  view = py3Dmol.view(width=800, height=600)
  view.addModel(open(pdb,'r').read(),'pdb')

  if color == "gray":
    view.setStyle({'cartoon':{}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  if show_glycan:
    HP = ['0AA', '0AB', '0AD', '0AE', '0AF', '0AU', '0BA', '0BB', '0BC', '0BD', '0BU', '0CA',
          '0CB', '0CD', '0CU', '0DA', '0DB', '0DD', '0DU', '0EA', '0EB', '0FA', '0FB', '0GA',
          '0GB', '0GL', '0HA', '0HB', '0JA', '0JB', '0JD', '0JU', '0KA', '0KB', '0LA', '0LB',
          '0MA', '0MB', '0NA', '0NB', '0OA', '0OB', '0PA', '0PB', '0PD', '0PU', '0QA', '0QB',
          '0RA', '0RB', '0RD', '0RU', '0SA', '0SB', '0TA', '0TB', '0TV', '0Tv', '0UA', '0UB',
          '0VA', '0VB', '0WA', '0WB', '0XA', '0XB', '0XD', '0XU', '0YA', '0YB', '0ZA', '0ZB',
          '0aA', '0aB', '0aD', '0aU', '0bA', '0bB', '0bC', '0bD', '0bU', '0cA', '0cB', '0cD',
          '0cU', '0dA', '0dB', '0dD', '0dR', '0dU', '0eA', '0eB', '0fA', '0fB', '0gA', '0gB',
          '0gL', '0hA', '0hB', '0jA', '0jB', '0jD', '0jU', '0kA', '0kB', '0lA', '0lB', '0mA',
          '0mB', '0nA', '0nB', '0oA', '0oB', '0pA', '0pB', '0pD', '0pU', '0qA', '0qB', '0rA',
          '0rB', '0rD', '0rU', '0sA', '0sB', '0tA', '0tB', '0tV', '0tv', '0uA', '0uB', '0vA',
          '0vB', '0wA', '0wB', '0xA', '0xB', '0xD', '0xU', '0yA', '0yB', '0zA', '0zB', '1AA',
          '1AB', '1AD', '1AU', '1BA', '1BB', '1BD', '1BU', '1CA', '1CB', '1CD', '1CU', '1DA',
          '1DB', '1DD', '1DU', '1EA', '1EB', '1FA', '1FB', '1GA', '1GB', '1HA', '1HB', '1JA',
          '1JB', '1JD', '1JU', '1KA', '1KB', '1LA', '1LB', '1MA', '1MB', '1NA', '1NB', '1OA',
          '1OB', '1PA', '1PB', '1PD', '1PU', '1QA', '1QB', '1RA', '1RB', '1RD', '1RU', '1TA',
          '1TB', '1TV', '1Tv', '1UA', '1UB', '1VA', '1VB', '1WA', '1WB', '1XA', '1XB', '1XD',
          '1XU', '1YA', '1YB', '1ZA', '1ZB', '1aA', '1aB', '1aD', '1aU', '1bA', '1bB', '1bD',
          '1bU', '1cA', '1cB', '1cD', '1cU', '1dA', '1dB', '1dD', '1dU', '1eA', '1eB', '1fA',
          '1fB', '1gA', '1gB', '1hA', '1hB', '1jA', '1jB', '1jD', '1jU', '1kA', '1kB', '1lA',
          '1lB', '1mA', '1mB', '1nA', '1nB', '1oA', '1oB', '1pA', '1pB', '1pD', '1pU', '1qA',
          '1qB', '1rA', '1rB', '1rD', '1rU', '1tA', '1tB', '1tV', '1tv', '1uA', '1uB', '1vA',
          '1vB', '1wA', '1wB', '1xA', '1xB', '1xD', '1xU', '1yA', '1yB', '1zA', '1zB', '2AA',
          '2AB', '2AD', '2AE', '2AF', '2AU', '2BA', '2BB', '2BD', '2BU', '2CA', '2CB', '2CD',
          '2CU', '2DA', '2DB', '2DD', '2DU', '2EA', '2EB', '2FA', '2FB', '2GA', '2GB', '2HA',
          '2HB', '2JA', '2JB', '2JD', '2JU', '2KA', '2KB', '2LA', '2LB', '2MA', '2MB', '2NA',
          '2NB', '2OA', '2OB', '2PA', '2PB', '2PD', '2PU', '2QA', '2QB', '2RA', '2RB', '2RD',
          '2RU', '2TA', '2TB', '2TV', '2Tv', '2UA', '2UB', '2XA', '2XB', '2XD', '2XU', '2ZA',
          '2ZB', '2aA', '2aB', '2aD', '2aU', '2bA', '2bB', '2bD', '2bU', '2cA', '2cB', '2cD',
          '2cU', '2dA', '2dB', '2dD', '2dU', '2eA', '2eB', '2fA', '2fB', '2gA', '2gB', '2hA',
          '2hB', '2jA', '2jB', '2jD', '2jU', '2kA', '2kB', '2lA', '2lB', '2mA', '2mB', '2nA',
          '2nB', '2oA', '2oB', '2pA', '2pB', '2pD', '2pU', '2qA', '2qB', '2rA', '2rB', '2rD',
          '2rU', '2tA', '2tB', '2tV', '2tv', '2uA', '2uB', '2xA', '2xB', '2xD', '2xU', '2zA',
          '2zB', '3AA', '3AB', '3AD', '3AU', '3BA', '3BB', '3BC', '3BD', '3BU', '3CA', '3CB',
          '3CD', '3CU', '3DA', '3DB', '3DD', '3DU', '3EA', '3EB', '3FA', '3FB', '3GA', '3GB',
          '3HA', '3HB', '3JA', '3JB', '3JD', '3JU', '3KA', '3KB', '3LA', '3LB', '3MA', '3MB',
          '3NA', '3NB', '3OA', '3OB', '3PA', '3PB', '3PD', '3PU', '3QA', '3QB', '3RA', '3RB',
          '3RD', '3RU', '3TA', '3TB', '3UA', '3UB', '3VA', '3VB', '3WA', '3WB', '3XA', '3XB',
          '3XD', '3XU', '3YA', '3YB', '3ZA', '3ZB', '3aA', '3aB', '3aD', '3aU', '3bA', '3bB',
          '3bC', '3bD', '3bU', '3cA', '3cB', '3cD', '3cU', '3dA', '3dB', '3dD', '3dR', '3dU',
          '3eA', '3eB', '3fA', '3fB', '3gA', '3gB', '3hA', '3hB', '3jA', '3jB', '3jD', '3jU',
          '3kA', '3kB', '3lA', '3lB', '3mA', '3mB', '3nA', '3nB', '3oA', '3oB', '3pA', '3pB',
          '3pD', '3pU', '3qA', '3qB', '3rA', '3rB', '3rD', '3rU', '3tA', '3tB', '3uA', '3uB',
          '3vA', '3vB', '3wA', '3wB', '3xA', '3xB', '3xD', '3xU', '3yA', '3yB', '3zA', '3zB',
          '4AA', '4AB', '4AE', '4AF', '4BA', '4BB', '4BD', '4BU', '4CA', '4CB', '4CD', '4CU',
          '4DA', '4DB', '4EA', '4EB', '4FA', '4FB', '4GA', '4GB', '4GL', '4HA', '4HB', '4JA',
          '4JB', '4JD', '4JU', '4KA', '4KB', '4LA', '4LB', '4MA', '4MB', '4NA', '4NB', '4OA',
          '4OB', '4PA', '4PB', '4PD', '4PU', '4QA', '4QB', '4RA', '4RB', '4SA', '4SB', '4TA',
          '4TB', '4TV', '4Tv', '4UA', '4UB', '4VA', '4VB', '4WA', '4WB', '4XA', '4XB', '4YA',
          '4YB', '4ZA', '4ZB', '4aA', '4aB', '4bA', '4bB', '4bD', '4bU', '4cA', '4cB', '4cD',
          '4cU', '4dA', '4dB', '4dR', '4eA', '4eB', '4fA', '4fB', '4gA', '4gB', '4gL', '4hA',
          '4hB', '4jA', '4jB', '4jD', '4jU', '4kA', '4kB', '4lA', '4lB', '4mA', '4mB', '4nA',
          '4nB', '4oA', '4oB', '4pA', '4pB', '4pD', '4pU', '4qA', '4qB', '4rA', '4rB', '4sA',
          '4sB', '4tA', '4tB', '4tV', '4tv', '4uA', '4uB', '4vA', '4vB', '4wA', '4wB', '4xA',
          '4xB', '4yA', '4yB', '4zA', '4zB', '5AD', '5AU', '5BA', '5BB', '5CA', '5CB', '5DD',
          '5DU', '5JA', '5JB', '5PA', '5PB', '5RD', '5RU', '5XD', '5XU', '5aD', '5aU', '5bA',
          '5bB', '5cA', '5cB', '5dD', '5dU', '5jA', '5jB', '5pA', '5pB', '5rD', '5rU', '5xD',
          '5xU', '6BD', '6BU', '6CD', '6CU', '6EA', '6EB', '6GA', '6GB', '6JD', '6JU', '6KA',
          '6KB', '6LA', '6LB', '6MA', '6MB', '6NA', '6NB', '6PD', '6PU', '6TA', '6TB', '6VA',
          '6VB', '6WA', '6WB', '6YA', '6YB', '6bD', '6bU', '6cD', '6cU', '6eA', '6eB', '6gA',
          '6gB', '6jD', '6jU', '6kA', '6kB', '6lA', '6lB', '6mA', '6mB', '6nA', '6nB', '6pD',
          '6pU', '6tA', '6tB', '6vA', '6vB', '6wA', '6wB', '6yA', '6yB', '7GL', '7SA', '7SB',
          '7gL', '7sA', '7sB', '8GL', '8SA', '8SB', '8gL', '8sA', '8sB', '9GL', '9SA', '9SB',
          '9gL', '9sA', '9sB', 'ACX', 'AGL', 'ASA', 'ASB', 'AgL', 'AsA', 'AsB', 'BGL', 'BSA',
          'BSB', 'BgL', 'BsA', 'BsB', 'CA2', 'CGL', 'CSA', 'CSB', 'CgL', 'CsA', 'CsB', 'DGL',
          'DSA', 'DSB', 'DgL', 'DsA', 'DsB', 'EGL', 'ESA', 'ESB', 'EgL', 'EsA', 'EsB', 'FGL',
          'FSA', 'FSB', 'FgL', 'FsA', 'FsB', 'GGL', 'GSA', 'GSB', 'GgL', 'GsA', 'GsB', 'HGL',
          'HSA', 'HSB', 'HgL', 'HsA', 'HsB', 'IGL', 'ISA', 'ISB', 'IgL', 'IsA', 'IsB', 'JGL',
          'JSA', 'JSB', 'JgL', 'JsA', 'JsB', 'KGL', 'KSA', 'KSB', 'KgL', 'KsA', 'KsB', 'MEX',
          'NLN', 'OLS', 'OLT', 'OME', 'PEA', 'PEB', 'PGA', 'PGB', 'PKA', 'PKB', 'PLA', 'PLB',
          'PMA', 'PMB', 'PNA', 'PNB', 'PTA', 'PTB', 'PeA', 'PeB', 'PgA', 'PgB', 'PkA', 'PkB',
          'PlA', 'PlB', 'PmA', 'PmB', 'PnA', 'PnB', 'PtA', 'PtB', 'QBD', 'QBU', 'QCD', 'QCU',
          'QEA', 'QEB', 'QGA', 'QGB', 'QJD', 'QJU', 'QKA', 'QKB', 'QLA', 'QLB', 'QMA', 'QMB',
          'QNA', 'QNB', 'QPD', 'QPU', 'QTA', 'QTB', 'QVA', 'QVB', 'QWA', 'QWB', 'QYA', 'QYB',
          'QbD', 'QbU', 'QcD', 'QcU', 'QeA', 'QeB', 'QgA', 'QgB', 'QjD', 'QjU', 'QkA', 'QkB',
          'QlA', 'QlB', 'QmA', 'QmB', 'QnA', 'QnB', 'QpD', 'QpU', 'QtA', 'QtB', 'QvA', 'QvB',
          'QwA', 'QwB', 'QyA', 'QyB', 'REA', 'REB', 'RGA', 'RGB', 'RKA', 'RKB', 'RLA', 'RLB',
          'RMA', 'RMB', 'RNA', 'RNB', 'ROH', 'RTA', 'RTB', 'ReA', 'ReB', 'RgA', 'RgB', 'RkA',
          'RkB', 'RlA', 'RlB', 'RmA', 'RmB', 'RnA', 'RnB', 'RtA', 'RtB', 'SEA', 'SEB', 'SGA',
          'SGB', 'SKA', 'SKB', 'SLA', 'SLB', 'SMA', 'SMB', 'SNA', 'SNB', 'SO3', 'STA', 'STB',
          'SeA', 'SeB', 'SgA', 'SgB', 'SkA', 'SkB', 'SlA', 'SlB', 'SmA', 'SmB', 'SnA', 'SnB',
          'StA', 'StB', 'TAA', 'TAB', 'TBT', 'TDA', 'TDB', 'TEA', 'TEB', 'TFA', 'TFB', 'TGA',
          'TGB', 'THA', 'THB', 'TKA', 'TKB', 'TLA', 'TLB', 'TMA', 'TMB', 'TNA', 'TNB', 'TOA',
          'TOB', 'TQA', 'TQB', 'TRA', 'TRB', 'TTA', 'TTB', 'TUA', 'TUB', 'TXA', 'TXB', 'TZA',
          'TZB', 'TaA', 'TaB', 'TdA', 'TdB', 'TeA', 'TeB', 'TfA', 'TfB', 'TgA', 'TgB', 'ThA',
          'ThB', 'TkA', 'TkB', 'TlA', 'TlB', 'TmA', 'TmB', 'TnA', 'TnB', 'ToA', 'ToB', 'TqA',
          'TqB', 'TrA', 'TrB', 'TtA', 'TtB', 'TuA', 'TuB', 'TxA', 'TxB', 'TzA', 'TzB', 'UBD',
          'UBU', 'UCD', 'UCU', 'UEA', 'UEB', 'UGA', 'UGB', 'UJD', 'UJU', 'UKA', 'UKB', 'ULA',
          'ULB', 'UMA', 'UMB', 'UNA', 'UNB', 'UPD', 'UPU', 'UTA', 'UTB', 'UVA', 'UVB', 'UWA',
          'UWB', 'UYA', 'UYB', 'UbD', 'UbU', 'UcD', 'UcU', 'UeA', 'UeB', 'UgA', 'UgB', 'UjD',
          'UjU', 'UkA', 'UkB', 'UlA', 'UlB', 'UmA', 'UmB', 'UnA', 'UnB', 'UpD', 'UpU', 'UtA',
          'UtB', 'UvA', 'UvB', 'UwA', 'UwB', 'UyA', 'UyB', 'VBD', 'VBU', 'VCD', 'VCU', 'VEA',
          'VEB', 'VGA', 'VGB', 'VJD', 'VJU', 'VKA', 'VKB', 'VLA', 'VLB', 'VMA', 'VMB', 'VNA',
          'VNB', 'VPD', 'VPU', 'VTA', 'VTB', 'VVA', 'VVB', 'VWA', 'VWB', 'VYA', 'VYB', 'VbD',
          'VbU', 'VcD', 'VcU', 'VeA', 'VeB', 'VgA', 'VgB', 'VjD', 'VjU', 'VkA', 'VkB', 'VlA',
          'VlB', 'VmA', 'VmB', 'VnA', 'VnB', 'VpD', 'VpU', 'VtA', 'VtB', 'VvA', 'VvB', 'VwA',
          'VwB', 'VyA', 'VyB', 'WAA', 'WAB', 'WBA', 'WBB', 'WBD', 'WBU', 'WCA', 'WCB', 'WCD',
          'WCU', 'WDA', 'WDB', 'WEA', 'WEB', 'WFA', 'WFB', 'WGA', 'WGB', 'WHA', 'WHB', 'WJA',
          'WJB', 'WJD', 'WJU', 'WKA', 'WKB', 'WLA', 'WLB', 'WMA', 'WMB', 'WNA', 'WNB', 'WOA',
          'WOB', 'WPA', 'WPB', 'WPD', 'WPU', 'WQA', 'WQB', 'WRA', 'WRB', 'WTA', 'WTB', 'WUA',
          'WUB', 'WVA', 'WVB', 'WWA', 'WWB', 'WXA', 'WXB', 'WYA', 'WYB', 'WZA', 'WZB', 'WaA',
          'WaB', 'WbA', 'WbB', 'WbD', 'WbU', 'WcA', 'WcB', 'WcD', 'WcU', 'WdA', 'WdB', 'WdR',
          'WeA', 'WeB', 'WfA', 'WfB', 'WgA', 'WgB', 'WhA', 'WhB', 'WjA', 'WjB', 'WjD', 'WjU',
          'WkA', 'WkB', 'WlA', 'WlB', 'WmA', 'WmB', 'WnA', 'WnB', 'WoA', 'WoB', 'WpA', 'WpB',
          'WpD', 'WpU', 'WqA', 'WqB', 'WrA', 'WrB', 'WtA', 'WtB', 'WuA', 'WuB', 'WvA', 'WvB',
          'WwA', 'WwB', 'WxA', 'WxB', 'WyA', 'WyB', 'WzA', 'WzB', 'XEA', 'XEB', 'XGA', 'XGB',
          'XKA', 'XKB', 'XLA', 'XLB', 'XMA', 'XMB', 'XNA', 'XNB', 'XTA', 'XTB', 'XeA', 'XeB',
          'XgA', 'XgB', 'XkA', 'XkB', 'XlA', 'XlB', 'XmA', 'XmB', 'XnA', 'XnB', 'XtA', 'XtB',
          'YAA', 'YAB', 'YAF', 'YDA', 'YDB', 'YEA', 'YEB', 'YFA', 'YFB', 'YGA', 'YGB', 'YGa',
          'YHA', 'YHB', 'YKA', 'YKB', 'YLA', 'YLB', 'YMA', 'YMB', 'YNA', 'YNB', 'YOA', 'YOB',
          'YQA', 'YQB', 'YRA', 'YRB', 'YTA', 'YTB', 'YTV', 'YTv', 'YUA', 'YUB', 'YXA', 'YXB',
          'YZA', 'YZB', 'YaA', 'YaB', 'YdA', 'YdB', 'YeA', 'YeB', 'YfA', 'YfB', 'YgA', 'YgB',
          'YhA', 'YhB', 'YkA', 'YkB', 'YlA', 'YlB', 'YmA', 'YmB', 'YnA', 'YnB', 'YoA', 'YoB',
          'YqA', 'YqB', 'YrA', 'YrB', 'YtA', 'YtB', 'YtV', 'Ytv', 'YuA', 'YuB', 'YxA', 'YxB',
          'YzA', 'YzB', 'ZAA', 'ZAB', 'ZAD', 'ZAU', 'ZDA', 'ZDB', 'ZDD', 'ZDU', 'ZEA', 'ZEB',
          'ZFA', 'ZFB', 'ZGA', 'ZGB', 'ZHA', 'ZHB', 'ZKA', 'ZKB', 'ZLA', 'ZLB', 'ZMA', 'ZMB',
          'ZNA', 'ZNB', 'ZOA', 'ZOB', 'ZOL', 'ZOL', 'ZQA', 'ZQB', 'ZRA', 'ZRB', 'ZRD', 'ZRU',
          'ZTA', 'ZTB', 'ZUA', 'ZUB', 'ZXA', 'ZXB', 'ZXD', 'ZXU', 'ZZA', 'ZZB', 'ZaA', 'ZaB',
          'ZaD', 'ZaU', 'ZdA', 'ZdB', 'ZdD', 'ZdU', 'ZeA', 'ZeB', 'ZfA', 'ZfB', 'ZgA', 'ZgB',
          'ZhA', 'ZhB', 'ZkA', 'ZkB', 'ZlA', 'ZlB', 'ZmA', 'ZmB', 'ZnA', 'ZnB', 'ZoA', 'ZoB',
          'ZqA', 'ZqB', 'ZrA', 'ZrB', 'ZrD', 'ZrU', 'ZtA', 'ZtB', 'ZuA', 'ZuB', 'ZxA', 'ZxB',
          'ZxD', 'ZxU', 'ZzA', 'ZzB', 'HYP', 'NLN', 'OLP', 'OLS', 'OLT', 'CHYP', 'CNLN', 'COLP',
          'COLS', 'COLT', 'NHYP', 'NNLN', 'NOLP', 'NOLS', 'NOLT']
    view.addStyle({'and':[{'resn':HP}]},
                       {'stick':{'colorscheme':'greenCarbon','radius':0.3}})
    view.setViewStyle({'style':'outline','color':'black','width':0.1})

  if show_box:
    view.addSurface(py3Dmol.SAS, {'opacity': box_opacity, 'color':'white'})

  view.zoomTo()
  return view


show_pdb(show_sidechains, show_mainchains, show_glycan, show_box, color).show()

---
---
# **Equilibrating the simulation box**

Proper MD equilibration protocol is designed to equilibrate both temperature and pressure throughout the simulation box while preserving the protein experimental conformation. In addition, we also allow the solvent to accomodate around the protein, creating proper solvation layers.

Below, we will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restraint protein heavy-atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

After you are done, you can run the next 2 cells to equilibrate your system.

In [ ]:
#@title ### **Parameters for MD Equilibration protocol:**

# remove whitespaces
Jobname = 'equil' #@param {type:"string"}

Minimization_steps = "1000" #@param ["1000", "5000", "10000", "20000", "50000", "100000"]

#@markdown Simulation time (in nanoseconds) and integration time (in femtoseconds):
Time = "5" #@param {type:"string"}
stride_time_eq = Time
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_eq = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = 298 #@param {type:"string"}
temperature_eq = Temperature
Pressure = 1 #@param {type:"string"}
pressure_eq = Pressure

#@markdown Position restraints force constant (in kJ/mol):
Force_constant = 500 #@param {type:"slider", min:0, max:2000, step:100}

#@markdown Frequency to write the trajectory file (in picoseconds):

Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_eq = Write_the_trajectory
#@markdown Frequency to write the log file (in picoseconds):

Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_eq = Write_the_log


#@markdown ---


In [ ]:
#@title **Runs an Equilibration MD simulation (NPT ensemble)**
#@markdown Now, let's equilibrate our system!

###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import pytraj as pt

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, Jobname)
coordinatefile = os.path.join(workDir, "SYS.crd")
pdbfile = os.path.join(workDir, "SYS.pdb")
topologyfile = os.path.join(workDir, "SYS.prmtop")

time_ps = float(Time)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def restraints(system, crd, fc, restraint_array):

	boxlx = system.getDefaultPeriodicBoxVectors()[0][0].value_in_unit(nanometers)
	boxly = system.getDefaultPeriodicBoxVectors()[1][1].value_in_unit(nanometers)
	boxlz = system.getDefaultPeriodicBoxVectors()[2][2].value_in_unit(nanometers)

	if fc > 0:
		# positional restraints for all heavy-atoms
		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
		posresPROT.addPerParticleParameter('k')
		posresPROT.addPerParticleParameter('x0')
		posresPROT.addPerParticleParameter('y0')
		posresPROT.addPerParticleParameter('z0')

		for atom1 in restraint_array:
			atom1 = int(atom1)

			xpos  = crd.positions[atom1].value_in_unit(nanometers)[0]
			ypos  = crd.positions[atom1].value_in_unit(nanometers)[1]
			zpos  = crd.positions[atom1].value_in_unit(nanometers)[2]

			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])

		system.addForce(posresPROT)

	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)

print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                           constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)

print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
pt_system = pt.iterload(coordinatefile, topologyfile)
pt_topology = pt_system.top
restraint_array = pt.select_atoms('!(:H*) & !(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+)', pt_topology)

system = restraints(system, inpcrd, restraint_fc, restraint_array)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

print("\t- Energy minimization: " + str(Minimization_steps) + " steps")
simulation.minimizeEnergy(tolerance=10*kilojoule/mole, maxIterations=int(Minimization_steps))
print("\t-> Potential Energy = " + str(simulation.context.getState(getEnergy=True).getPotentialEnergy()))

print("\t- Setting initial velocities...")
simulation.context.setVelocitiesToTemperature(temperature)

#############################################
# Running Equilibration on NPT ensemble

dcd_file = jobname + ".dcd"
log_file = jobname + ".log"
rst_file = jobname + ".rst"
prv_rst_file = jobname + ".rst"
pdb_file = jobname + ".pdb"

# Creating a trajectory file and reporters
dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (nsteps) + nsavcrd
dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

---
---
# **Running a Production MD simulation**

Finally, we will proceed with the Production simulation itself using the equilibrated system coordinates as input structure.

Note that we will use here a *.rst state file* , which contains atomic velocities and positions from the last frame of the equilibration simulation, guaranteeing that our production simulation begins from a thermodynamically equilibrated system.

Another important information here is the **Number_of_strides** and the **Stride_Time**. In this notebook, we simulate a defined number of *strides*, so the **simulation time = Number_of_strides*Stride_Time**. For example, we can simulate 100ns by setting *Number_of_strides=10* and *Stride_Time=10 ns*.

**Important: at the end of the Production simulation, we concatenate all strides to create a complete trajectory file which can be visualized and analyzed**

The idea behind this approach is to make use of the intermitent 12h/24h period that Google Colab allows us to use its GPUs.

In [ ]:
#@markdown ### **Provide input file names below:**

Equilibrated_PDB = 'equil.pdb' #@param {type:"string"}
State_file = 'equil.rst' #@param {type:"string"}

#@markdown ---
#@markdown ### **Parameters for MD Production protocol:**


# remove whitespaces
Jobname = 'prod' #@param {type:"string"}

#@markdown Simulation time (in nanoseconds), number of strides (integers) and integration timestep (in femtoseconds):
Stride_Time = "10" #@param {type:"string"}
stride_time_prod = Stride_Time
Number_of_strides = "1" #@param {type:"string"}
nstride = Number_of_strides
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_prod = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = 298 #@param {type:"string"}
temperature_prod = Temperature
Pressure = 1 #@param {type:"string"}
pressure_prod = Pressure

#@markdown Frequency to write the trajectory file (in picoseconds):
Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_prod = Write_the_trajectory

#@markdown Frequency to write the log file (in picoseconds):
Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_prod = Write_the_log

#@markdown ---

In [ ]:
#@title **Runs a Production MD simulation (NPT ensemble) after equilibration**
#
###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, str(Jobname))
coordinatefile = os.path.join(workDir, "SYS.crd")
pdbfile = os.path.join(workDir, Equilibrated_PDB)
topologyfile = os.path.join(workDir, "SYS.prmtop")
equil_rst_file = os.path.join(workDir, State_file)


stride_time_ps = float(stride_time_prod)*1000
stride_time = float(stride_time_ps)*picosecond
nstride = int(Number_of_strides)
dt = int(dt_prod)*femtosecond
temperature = float(temperature_prod)*kelvin
savcrd_freq = int(write_the_trajectory_prod)*picosecond
print_freq  = int(write_the_log_prod)*picosecond

pressure	= float(pressure_prod)*bar

simulation_time = stride_time*nstride
nsteps  = int(stride_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
firststride = 1 # must be integer
#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(stride_time*nstride))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps*nstride))
print("\tNumber of strides = " + str(nstride) + " (" + str(stride_time) + " in each stride)")

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tSave checkpoint each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)


print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                           constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
	simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

#############################################
# Opening a loop of extension NSTRIDE to simulate the entire STRIDE_TIME*NSTRIDE
for n in range(1, nstride + 1):

	print("\n\n>>> Simulating Stride #" + str(n) + " <<<")

	dcd_file = jobname + "_" + str(n) + ".dcd"
	log_file = jobname + "_" + str(n) + ".log"
	rst_file = jobname + "_" + str(n) + ".rst"
	prv_rst_file = jobname + "_" + str(n-1) + ".rst"
	pdb_file = jobname + "_" + str(n) + ".pdb"

	if os.path.exists(rst_file):
		print("> Stride #" + str(n) + " finished (" + rst_file + " present). Moving to next stride... <")
		continue

	if n == 1:
		print("\n> Loading previous state from equilibration > " + equil_rst_file + " <")
		with open(equil_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

	else:
		print("> Loading previous state from > " + prv_rst_file + " <")
		with open(prv_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")


	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (currstep) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # first step should not be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=(nsteps*nstride), remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps... (Stride #" + str(n) + ")")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

In [ ]:
#@title **Concatenate and align the trajectory**
#@markdown **Important**: The **Google Drive Path**, **Jobname**, **Number of strides**, **stride time** and **trajectory saved frequency** should be the same you have been used to run your simulation in the previous steps.

import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

Google_Drive_Path = '/content/drive/MyDrive/protein_glycan' #@param {type:"string"}
workDir = Google_Drive_Path
Equilibrated_PDB = 'equil.pdb' #@param {type:"string"}
Jobname = "prod" #@param {type: "string"}
Skip = "1" #@param ["1", "2", "5", "10", "20", "50"]
stride_traj = Skip
Output_format = "dcd" #@param ["dcd", "pdb", "trr", "xtc"]
first_stride = "1" #@param {type:"string"}
Number_of_strides = "1" #@param {type:"string"}
nstride = int(Number_of_strides)
stride_time = "10" #@param {type:"string"}
trajectory_saved_frequency = "10" #@param ["10", "100", "200", "500", "1000"]
traj_save_freq = trajectory_saved_frequency
# Remove_waters = "no" #@param ["yes", "no"]
# stride_id_as_ref_for_alignment = "1" #@param {type: "string"}
output_prefix = first_stride+"-"+str(int(first_stride)+nstride-1)

stride_time_ps = float(stride_time)*1000
simulation_time_analysis = stride_time_ps*nstride
simulation_ns = float(stride_time)*int(Number_of_strides)
number_frames = int(simulation_time_analysis)/int(traj_save_freq)
number_frames_analysis = number_frames/int(stride_traj)


# nw_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_nw." + str(Output_format))
# nw_pdb = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
whole_pdb = os.path.join(workDir, str(Jobname) +  "_whole.pdb")
whole_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_whole." + str(Output_format))
template =  os.path.join(workDir, str(Jobname) + '_%s.dcd')
pdb = os.path.join(workDir, Equilibrated_PDB)

flist = [template % str(i) for i in range(int(first_stride), int(first_stride) + nstride)]
ref = [template % int(1)]

u1 = mda.Universe(os.path.join(workDir, "SYS.prmtop"), flist)
u2 = mda.Universe(os.path.join(workDir, "SYS.prmtop"), ref)

u2.trajectory[0] # set u2 to first frame

# print(rms.rmsd(u1.select_atoms('name CA').positions, u2.select_atoms('name CA').positions, superposition=False))

align.AlignTraj(u1, u2, select="not (resname WAT Na+ Cl- Mg+ K+) and (not element H)", in_memory=True).run()

with mda.Writer(whole_dcd, u1.select_atoms("all").n_atoms) as W:
  for ts in u1.trajectory[::int(Skip)]:
      W.write(u1.select_atoms("all"))
whole = u2.select_atoms("all")
whole.write(whole_pdb)
traj_dcd_check = os.path.exists(whole_dcd)
traj = whole_dcd
pdb_ref = whole_pdb

# nw = u1.select_atoms("not (resname WAT)")
# if Remove_waters == "yes":
#   with mda.Writer(nw_dcd, nw.n_atoms) as W:
#     for ts in u1.trajectory[::int(Skip)]:
#         W.write(nw)
#   not_waters = u2.select_atoms("not (resname WAT)")
#   not_waters.write(nw_pdb)
#   traj_dcd_check = os.path.exists(nw_dcd)
#   traj = nw_dcd
#   pdb_ref = nw_pdb
# else:
#   pass

traj_load = pt.load(traj, os.path.join(workDir, "SYS.prmtop"))
print(traj_load)

#mask_selection

mask_protein = "@CA"
pt_topology = traj_load.top
restraint_array1 = pt.select_atoms(':0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_glycan = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
mask_protein_glycan = mask_protein + "," + mask_glycan

if traj_dcd_check == True:
  print("Trajectory concatenated successfully! :-)")
else:
  print("ERROR: Check your inputs! ")

In [ ]:
#@title **Load, view and check the trajectory**
#@markdown This will take a few minutes. Another coffee would be great. :-)

#@markdown If you have a huge system, this step can explode the memory on Colab. Please, be careful.

import warnings
warnings.filterwarnings('ignore')

#py3dmol functions
class Atom(dict):
  def __init__(self, line):
    self["type"] = line[0:6].strip()
    self["idx"] = line[6:11].strip()
    self["name"] = line[12:16].strip()
    self["resname"] = line[17:20].strip()
    self["resid"] = int(int(line[22:26]))
    self["x"] = float(line[30:38])
    self["y"] = float(line[38:46])
    self["z"] = float(line[46:54])
    self["sym"] = line[76:78].strip()

  def __str__(self):
    line = list(" " * 80)
    line[0:6] = self["type"].ljust(6)
    line[6:11] = self["idx"].ljust(5)
    line[12:16] = self["name"].ljust(4)
    line[17:20] = self["resname"].ljust(3)
    line[22:26] = str(self["resid"]).ljust(4)
    line[30:38] = str(self["x"]).rjust(8)
    line[38:46] = str(self["y"]).rjust(8)
    line[46:54] = str(self["z"]).rjust(8)
    line[76:78] = self["sym"].rjust(2)
    return "".join(line) + "\n"

class Molecule(list):
  def __init__(self, file):
    for line in file:
      if "ATOM" in line or "HETATM" in line:
        self.append(Atom(line))

    def __str__(self):
      outstr = ""
      for at in self:
        outstr += str(at)
      return outstr

if number_frames_analysis > 10:
  stride_animation = number_frames_analysis/10
else:
  stride_animation = 1

u = mda.Universe(pdb_ref, traj)

# Write out frames for animation
protein = u.select_atoms("(not resname WAT Na+ Cl- Mg+ K+) and (not element H)")
i = 0
for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:
    if i > -1:
        with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
            W.write(protein)
    i = i + 1
# Load frames as molecules
molecules = []
for i in range(int(len(u.trajectory)/int(stride_animation))):
    with open('' + str(i) + '.pdb') as ifile:
        molecules.append(Molecule(ifile))

models = ""
for i in range(len(molecules)):
  models += "MODEL " + str(i) + "\n"
  for j,mol in enumerate(molecules[i]):
    models += str(mol)
  models += "ENDMDL\n"
#view.addModelsAsFrames(models)

# Animation
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
for i, at in enumerate(molecules[0]):
    default = {"cartoon": {'color': 'spectrum'}}
    view.setViewStyle({'style':'outline','color':'black','width':0.1})
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))
    view.setStyle({'model': -1, 'serial': i+1}, {'stick':{'radius':0.3}})
    # HP = ['protein']
    # view.setStyle({"model":-1,'and':[{'resn':HP}]},{'stick':{'radius':0.3}})
view.zoomTo()
view.animate({'loop': "forward"})
view.show()

---
---
# **Analysis**

Although visualizing your trajectory can be quite useful, sometimes you also want more quantitative data.

Analyses of MD trajectories vary a lot and we do not intend to cover it all here. However, one can make use of MDanalysis or PyTraj to easily analyze simulations.

Below, you can find a few examples of code snippets that can help you to shed some light on your simulation behavior.

In [ ]:
#@title **Compute hydrogen bonds between your molecule and waters**

Selection = "Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  pt_topology = traj_load.top
  restraint_array1 = pt.select_atoms('!(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+) & !:0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
  first_atom1 = restraint_array1[0]
  last_atom1 = restraint_array1[-1]
  mask_protein_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
  mask_end = mask_protein_hbond
elif Selection == "Protein+Glycan":
  pt_topology = traj_load.top
  restraint_array1 = pt.select_atoms('!(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+)', pt_topology)
  first_atom1 = restraint_array1[0]
  last_atom1 = restraint_array1[-1]
  mask_protein_glycan_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
  mask_end = mask_protein_glycan_hbond
else:
  pt_topology = traj_load.top
  restraint_array1 = pt.select_atoms(':0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
  first_atom1 = restraint_array1[0]
  last_atom1 = restraint_array1[-1]
  mask_glycan_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
  mask_end = mask_glycan_hbond

#@markdown **Provide output file names below:**
Output_name = 'hbond_solv' #@param {type:"string"}
mask_hbond1 = 'acceptormask ' + mask_end + ' donormask :WAT'
mask_hbond2 = 'acceptormask :WAT' + ' donormask ' + mask_end


hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
hb2 = pt.hbond(traj_load, options=mask_hbond2, dtype='dict', distance =3.5)

hb_end1 = hb1['total_solute_hbonds']
hb_end2 = hb2['total_solute_hbonds']
hb_total = hb_end1 + hb_end2
time = len(hb_total)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)
# hb.keys()
# print(hb_total)
# hb_end_mean = mean(hb_total)
# hb_end_stdev = stdev(hb_total)
# print("Hydrogen Bonds Average = " + str("{:.2f}".format(hb_end_mean)) + " \u00B1 " + str("{:.2f}".format(hb_end_stdev)))

# Plotting:
ax = plt.plot(time_array, hb_total, alpha=0.6, color = 'black', linewidth = 1.0)
plt.xlim(0, simulation_ns)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Hydrogen Bonds", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(hb_total)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute intramolecular hydrogen bonds**

Selection_1 = "Protein" #@param ["Protein", "Glycan"]
Selection_2 = "Glycan" #@param ["Protein", "Glycan"]

pt_topology = traj_load.top
restraint_array1 = pt.select_atoms('!(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+) & !:0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_protein_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)

pt_topology = traj_load.top
restraint_array1 = pt.select_atoms(':0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_glycan_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)

if Selection_1 == "Protein"  and Selection_2 == "Protein":
  mask_end = mask_protein_hbond
  mask_hbond1 = 'acceptormask ' + mask_end + ' donormask ' + mask_end
  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb = hb1['total_solute_hbonds']
  # print(mask_end)
elif Selection_1 == "Glycan"  and Selection_2 == "Glycan":
  mask_end = mask_glycan_hbond
  mask_hbond1 = 'acceptormask ' + mask_end + ' donormask ' + mask_end
  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb = hb1['total_solute_hbonds']
  # print(mask_end)
else:
  mask_end1 = mask_protein_hbond
  mask_end2 = mask_glycan_hbond
  mask_hbond1 = 'acceptormask ' + mask_end1 + ' donormask ' + mask_end2
  mask_hbond2 = 'acceptormask ' + mask_end2 + ' donormask ' + mask_end1

  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb2 = pt.hbond(traj_load, options=mask_hbond2, dtype='dict', distance =3.5)

  hb_end1 = hb1['total_solute_hbonds']
  hb_end2 = hb2['total_solute_hbonds']
  hb = hb_end1 + hb_end2
  # print(mask_end1,mask_end2)

#@markdown **Provide output file names below:**
Output_name = 'hbond_intra' #@param {type:"string"}

time = len(hb)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)
# hb.keys()
# print(hb_total)
# hb_end_mean = mean(hb_total)
# hb_end_stdev = stdev(hb_total)
# print("Hydrogen Bonds Average = " + str("{:.2f}".format(hb_end_mean)) + " \u00B1 " + str("{:.2f}".format(hb_end_stdev)))

# Plotting:
ax = plt.plot(time_array, hb, alpha=0.6, color = 'black', linewidth = 1.0)
plt.xlim(0, simulation_ns)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Hydrogen Bonds", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(hb_total)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute solvent-accessible surface area (SASA)**
#@markdown **Provide output file names below:**

Selection = "Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]
pt_topology = traj_load.top
restraint_array1 = pt.select_atoms('!(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+) & !:0AA,0AB,0AD,0AE,0AF,0AU,0BA,0BB,0BC,0BD,0BU,0CA,0CB,0CD,0CU,0DA,0DB,0DD,0DU,0EA,0EB,0FA,0FB,0GA,0GB,0GL,0HA,0HB,0JA,0JB,0JD,0JU,0KA,0KB,0LA,0LB,0MA,0MB,0NA,0NB,0OA,0OB,0PA,0PB,0PD,0PU,0QA,0QB,0RA,0RB,0RD,0RU,0SA,0SB,0TA,0TB,0TV,0Tv,0UA,0UB,0VA,0VB,0WA,0WB,0XA,0XB,0XD,0XU,0YA,0YB,0ZA,0ZB,0aA,0aB,0aD,0aU,0bA,0bB,0bC,0bD,0bU,0cA,0cB,0cD,0cU,0dA,0dB,0dD,0dR,0dU,0eA,0eB,0fA,0fB,0gA,0gB,0gL,0hA,0hB,0jA,0jB,0jD,0jU,0kA,0kB,0lA,0lB,0mA,0mB,0nA,0nB,0oA,0oB,0pA,0pB,0pD,0pU,0qA,0qB,0rA,0rB,0rD,0rU,0sA,0sB,0tA,0tB,0tV,0tv,0uA,0uB,0vA,0vB,0wA,0wB,0xA,0xB,0xD,0xU,0yA,0yB,0zA,0zB,1AA,1AB,1AD,1AU,1BA,1BB,1BD,1BU,1CA,1CB,1CD,1CU,1DA,1DB,1DD,1DU,1EA,1EB,1FA,1FB,1GA,1GB,1HA,1HB,1JA,1JB,1JD,1JU,1KA,1KB,1LA,1LB,1MA,1MB,1NA,1NB,1OA,1OB,1PA,1PB,1PD,1PU,1QA,1QB,1RA,1RB,1RD,1RU,1TA,1TB,1TV,1Tv,1UA,1UB,1VA,1VB,1WA,1WB,1XA,1XB,1XD,1XU,1YA,1YB,1ZA,1ZB,1aA,1aB,1aD,1aU,1bA,1bB,1bD,1bU,1cA,1cB,1cD,1cU,1dA,1dB,1dD,1dU,1eA,1eB,1fA,1fB,1gA,1gB,1hA,1hB,1jA,1jB,1jD,1jU,1kA,1kB,1lA,1lB,1mA,1mB,1nA,1nB,1oA,1oB,1pA,1pB,1pD,1pU,1qA,1qB,1rA,1rB,1rD,1rU,1tA,1tB,1tV,1tv,1uA,1uB,1vA,1vB,1wA,1wB,1xA,1xB,1xD,1xU,1yA,1yB,1zA,1zB,2AA,2AB,2AD,2AE,2AF,2AU,2BA,2BB,2BD,2BU,2CA,2CB,2CD,2CU,2DA,2DB,2DD,2DU,2EA,2EB,2FA,2FB,2GA,2GB,2HA,2HB,2JA,2JB,2JD,2JU,2KA,2KB,2LA,2LB,2MA,2MB,2NA,2NB,2OA,2OB,2PA,2PB,2PD,2PU,2QA,2QB,2RA,2RB,2RD,2RU,2TA,2TB,2TV,2Tv,2UA,2UB,2XA,2XB,2XD,2XU,2ZA,2ZB,2aA,2aB,2aD,2aU,2bA,2bB,2bD,2bU,2cA,2cB,2cD,2cU,2dA,2dB,2dD,2dU,2eA,2eB,2fA,2fB,2gA,2gB,2hA,2hB,2jA,2jB,2jD,2jU,2kA,2kB,2lA,2lB,2mA,2mB,2nA,2nB,2oA,2oB,2pA,2pB,2pD,2pU,2qA,2qB,2rA,2rB,2rD,2rU,2tA,2tB,2tV,2tv,2uA,2uB,2xA,2xB,2xD,2xU,2zA,2zB,3AA,3AB,3AD,3AU,3BA,3BB,3BC,3BD,3BU,3CA,3CB,3CD,3CU,3DA,3DB,3DD,3DU,3EA,3EB,3FA,3FB,3GA,3GB,3HA,3HB,3JA,3JB,3JD,3JU,3KA,3KB,3LA,3LB,3MA,3MB,3NA,3NB,3OA,3OB,3PA,3PB,3PD,3PU,3QA,3QB,3RA,3RB,3RD,3RU,3TA,3TB,3UA,3UB,3VA,3VB,3WA,3WB,3XA,3XB,3XD,3XU,3YA,3YB,3ZA,3ZB,3aA,3aB,3aD,3aU,3bA,3bB,3bC,3bD,3bU,3cA,3cB,3cD,3cU,3dA,3dB,3dD,3dR,3dU,3eA,3eB,3fA,3fB,3gA,3gB,3hA,3hB,3jA,3jB,3jD,3jU,3kA,3kB,3lA,3lB,3mA,3mB,3nA,3nB,3oA,3oB,3pA,3pB,3pD,3pU,3qA,3qB,3rA,3rB,3rD,3rU,3tA,3tB,3uA,3uB,3vA,3vB,3wA,3wB,3xA,3xB,3xD,3xU,3yA,3yB,3zA,3zB,4AA,4AB,4AE,4AF,4BA,4BB,4BD,4BU,4CA,4CB,4CD,4CU,4DA,4DB,4EA,4EB,4FA,4FB,4GA,4GB,4GL,4HA,4HB,4JA,4JB,4JD,4JU,4KA,4KB,4LA,4LB,4MA,4MB,4NA,4NB,4OA,4OB,4PA,4PB,4PD,4PU,4QA,4QB,4RA,4RB,4SA,4SB,4TA,4TB,4TV,4Tv,4UA,4UB,4VA,4VB,4WA,4WB,4XA,4XB,4YA,4YB,4ZA,4ZB,4aA,4aB,4bA,4bB,4bD,4bU,4cA,4cB,4cD,4cU,4dA,4dB,4dR,4eA,4eB,4fA,4fB,4gA,4gB,4gL,4hA,4hB,4jA,4jB,4jD,4jU,4kA,4kB,4lA,4lB,4mA,4mB,4nA,4nB,4oA,4oB,4pA,4pB,4pD,4pU,4qA,4qB,4rA,4rB,4sA,4sB,4tA,4tB,4tV,4tv,4uA,4uB,4vA,4vB,4wA,4wB,4xA,4xB,4yA,4yB,4zA,4zB,5AD,5AU,5BA,5BB,5CA,5CB,5DD,5DU,5JA,5JB,5PA,5PB,5RD,5RU,5XD,5XU,5aD,5aU,5bA,5bB,5cA,5cB,5dD,5dU,5jA,5jB,5pA,5pB,5rD,5rU,5xD,5xU,6BD,6BU,6CD,6CU,6EA,6EB,6GA,6GB,6JD,6JU,6KA,6KB,6LA,6LB,6MA,6MB,6NA,6NB,6PD,6PU,6TA,6TB,6VA,6VB,6WA,6WB,6YA,6YB,6bD,6bU,6cD,6cU,6eA,6eB,6gA,6gB,6jD,6jU,6kA,6kB,6lA,6lB,6mA,6mB,6nA,6nB,6pD,6pU,6tA,6tB,6vA,6vB,6wA,6wB,6yA,6yB,7GL,7SA,7SB,7gL,7sA,7sB,8GL,8SA,8SB,8gL,8sA,8sB,9GL,9SA,9SB,9gL,9sA,9sB,ACX,AGL,ASA,ASB,AgL,AsA,AsB,BGL,BSA,BSB,BgL,BsA,BsB,CA2,CGL,CSA,CSB,CgL,CsA,CsB,DGL,DSA,DSB,DgL,DsA,DsB,EGL,ESA,ESB,EgL,EsA,EsB,FGL,FSA,FSB,FgL,FsA,FsB,GGL,GSA,GSB,GgL,GsA,GsB,HGL,HSA,HSB,HgL,HsA,HsB,IGL,ISA,ISB,IgL,IsA,IsB,JGL,JSA,JSB,JgL,JsA,JsB,KGL,KSA,KSB,KgL,KsA,KsB,MEX,NLN,OLS,OLT,OME,PEA,PEB,PGA,PGB,PKA,PKB,PLA,PLB,PMA,PMB,PNA,PNB,PTA,PTB,PeA,PeB,PgA,PgB,PkA,PkB,PlA,PlB,PmA,PmB,PnA,PnB,PtA,PtB,QBD,QBU,QCD,QCU,QEA,QEB,QGA,QGB,QJD,QJU,QKA,QKB,QLA,QLB,QMA,QMB,QNA,QNB,QPD,QPU,QTA,QTB,QVA,QVB,QWA,QWB,QYA,QYB,QbD,QbU,QcD,QcU,QeA,QeB,QgA,QgB,QjD,QjU,QkA,QkB,QlA,QlB,QmA,QmB,QnA,QnB,QpD,QpU,QtA,QtB,QvA,QvB,QwA,QwB,QyA,QyB,REA,REB,RGA,RGB,RKA,RKB,RLA,RLB,RMA,RMB,RNA,RNB,ROH,RTA,RTB,ReA,ReB,RgA,RgB,RkA,RkB,RlA,RlB,RmA,RmB,RnA,RnB,RtA,RtB,SEA,SEB,SGA,SGB,SKA,SKB,SLA,SLB,SMA,SMB,SNA,SNB,SO3,STA,STB,SeA,SeB,SgA,SgB,SkA,SkB,SlA,SlB,SmA,SmB,SnA,SnB,StA,StB,TAA,TAB,TBT,TDA,TDB,TEA,TEB,TFA,TFB,TGA,TGB,THA,THB,TKA,TKB,TLA,TLB,TMA,TMB,TNA,TNB,TOA,TOB,TQA,TQB,TRA,TRB,TTA,TTB,TUA,TUB,TXA,TXB,TZA,TZB,TaA,TaB,TdA,TdB,TeA,TeB,TfA,TfB,TgA,TgB,ThA,ThB,TkA,TkB,TlA,TlB,TmA,TmB,TnA,TnB,ToA,ToB,TqA,TqB,TrA,TrB,TtA,TtB,TuA,TuB,TxA,TxB,TzA,TzB,UBD,UBU,UCD,UCU,UEA,UEB,UGA,UGB,UJD,UJU,UKA,UKB,ULA,ULB,UMA,UMB,UNA,UNB,UPD,UPU,UTA,UTB,UVA,UVB,UWA,UWB,UYA,UYB,UbD,UbU,UcD,UcU,UeA,UeB,UgA,UgB,UjD,UjU,UkA,UkB,UlA,UlB,UmA,UmB,UnA,UnB,UpD,UpU,UtA,UtB,UvA,UvB,UwA,UwB,UyA,UyB,VBD,VBU,VCD,VCU,VEA,VEB,VGA,VGB,VJD,VJU,VKA,VKB,VLA,VLB,VMA,VMB,VNA,VNB,VPD,VPU,VTA,VTB,VVA,VVB,VWA,VWB,VYA,VYB,VbD,VbU,VcD,VcU,VeA,VeB,VgA,VgB,VjD,VjU,VkA,VkB,VlA,VlB,VmA,VmB,VnA,VnB,VpD,VpU,VtA,VtB,VvA,VvB,VwA,VwB,VyA,VyB,WAA,WAB,WBA,WBB,WBD,WBU,WCA,WCB,WCD,WCU,WDA,WDB,WEA,WEB,WFA,WFB,WGA,WGB,WHA,WHB,WJA,WJB,WJD,WJU,WKA,WKB,WLA,WLB,WMA,WMB,WNA,WNB,WOA,WOB,WPA,WPB,WPD,WPU,WQA,WQB,WRA,WRB,WTA,WTB,WUA,WUB,WVA,WVB,WWA,WWB,WXA,WXB,WYA,WYB,WZA,WZB,WaA,WaB,WbA,WbB,WbD,WbU,WcA,WcB,WcD,WcU,WdA,WdB,WdR,WeA,WeB,WfA,WfB,WgA,WgB,WhA,WhB,WjA,WjB,WjD,WjU,WkA,WkB,WlA,WlB,WmA,WmB,WnA,WnB,WoA,WoB,WpA,WpB,WpD,WpU,WqA,WqB,WrA,WrB,WtA,WtB,WuA,WuB,WvA,WvB,WwA,WwB,WxA,WxB,WyA,WyB,WzA,WzB,XEA,XEB,XGA,XGB,XKA,XKB,XLA,XLB,XMA,XMB,XNA,XNB,XTA,XTB,XeA,XeB,XgA,XgB,XkA,XkB,XlA,XlB,XmA,XmB,XnA,XnB,XtA,XtB,YAA,YAB,YAF,YDA,YDB,YEA,YEB,YFA,YFB,YGA,YGB,YGa,YHA,YHB,YKA,YKB,YLA,YLB,YMA,YMB,YNA,YNB,YOA,YOB,YQA,YQB,YRA,YRB,YTA,YTB,YTV,YTv,YUA,YUB,YXA,YXB,YZA,YZB,YaA,YaB,YdA,YdB,YeA,YeB,YfA,YfB,YgA,YgB,YhA,YhB,YkA,YkB,YlA,YlB,YmA,YmB,YnA,YnB,YoA,YoB,YqA,YqB,YrA,YrB,YtA,YtB,YtV,Ytv,YuA,YuB,YxA,YxB,YzA,YzB,ZAA,ZAB,ZAD,ZAU,ZDA,ZDB,ZDD,ZDU,ZEA,ZEB,ZFA,ZFB,ZGA,ZGB,ZHA,ZHB,ZKA,ZKB,ZLA,ZLB,ZMA,ZMB,ZNA,ZNB,ZOA,ZOB,ZOL,ZOL,ZQA,ZQB,ZRA,ZRB,ZRD,ZRU,ZTA,ZTB,ZUA,ZUB,ZXA,ZXB,ZXD,ZXU,ZZA,ZZB,ZaA,ZaB,ZaD,ZaU,ZdA,ZdB,ZdD,ZdU,ZeA,ZeB,ZfA,ZfB,ZgA,ZgB,ZhA,ZhB,ZkA,ZkB,ZlA,ZlB,ZmA,ZmB,ZnA,ZnB,ZoA,ZoB,ZqA,ZqB,ZrA,ZrB,ZrD,ZrU,ZtA,ZtB,ZuA,ZuB,ZxA,ZxB,ZxD,ZxU,ZzA,ZzB,HYP,NLN,OLP,OLS,OLT,CHYP,CNLN,COLP,COLS,COLT,NHYP,NNLN,NOLP,NOLS,NOLT', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_protein_sasa = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
mask_protein_glycan_sasa = mask_protein_sasa + "," + mask_glycan

if Selection == "Protein":
  mask_end = mask_protein_sasa
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan_sasa
else:
  mask_end = mask_glycan

Output_name = 'sasa' #@param {type:"string"}

sasa = pt.molsurf(traj_load, mask_end)

time = len(sasa)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, sasa, alpha=1, color = 'darkcyan', linewidth = 1.0)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("SASA ($\AA^{2}$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(sasa)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot solvent-accessible surface area (SASA) as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'sasa_dist' #@param {type:"string"}

ax = sb.kdeplot(sasa, color="darkcyan", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('SASA ($\AA^{2}$)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute distance between the atoms**
#@markdown **Provide output file names below:**
Output_name = 'distance_atoms' #@param {type:"string"}
#@markdown **Type the number of atoms separated by commas and without spaces (1,2,3...):**
Selection_1 = '3' #@param {type:"string"}
Selection_2 = '12' #@param {type:"string"}

mask = "@" + str(Selection_1) + " @" + str(Selection_2)
dist = pt.distance(traj_load, mask)
print("Selected atoms = " + Selection_1 + ", "  + Selection_2 + "\n")
dist_mean = mean(dist)
dist_stdev = stdev(dist)
print("Distance Average = " + str("{:.2f}".format(dist_mean)) + " \u00B1 " + str("{:.2f}".format(dist_stdev)) + " Å")

time = len(dist)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, dist, alpha=1, color = 'magenta', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Distance [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(dist)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot distance as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'distance_dist' #@param {type:"string"}

ax = sb.kdeplot(dist, color="magenta", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Distance [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute specific dihedral angles**
#@markdown **Provide output file names below:**
Output_name = 'dihedral_angle' #@param {type:"string"}
#@markdown **Type the number of atoms:**
Atom_1 = '1' #@param {type:"string"}
Atom_2 = '2' #@param {type:"string"}
Atom_3 = '3' #@param {type:"string"}
Atom_4 = '4' #@param {type:"string"}

mask = "@" + str(Atom_1) + " @" + str(Atom_2) + " @" + str(Atom_3) + " @" + str(Atom_4)
dih = pt.dihedral(traj_load, mask)
print("Selected atoms = " + Atom_1 + ", " + Atom_2 + ", " + Atom_3 + ", " + Atom_4 + "\n")
dih_mean = mean(dih)
dih_stdev = stdev(dih)
print("Dihedral Angle Average = " + str("{:.2f}".format(dih_mean)) + " \u00B1 " + str("{:.2f}".format(dih_stdev)) + "°")

time = len(dih)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, dih, alpha=1, color = 'orange', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Dihedral Angle ($^\circ$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(dih)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot dihedral angle as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'dihedral_dist' #@param {type:"string"}

ax = sb.kdeplot(dih, color="orange", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Dihedral Angle ($^\circ$)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute RMSD**

Selection = "Protein+Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
else:
  mask_end = mask_glycan

#@markdown **Provide output file names below:**

Output_name = 'rmsd_protein' #@param {type:"string"}

rmsd = pt.rmsd(traj_load, ref = 0, mask = mask_end)

time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
ax = plt.plot(time_array, rmsd, alpha=0.6, color = 'blue', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsd)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot RMSD as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'rmsd_dist' #@param {type:"string"}

ax = sb.kdeplot(rmsd, color="blue", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute radius of gyration**

Selection = "Protein" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
else:
  mask_end = mask_glycan

#@markdown **Provide output file names below:**

Output_name = 'radius_gyration' #@param {type:"string"}

radgyr = pt.radgyr(traj_load, mask = mask_end)
time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

# Plotting:
plt.plot(time_array, radgyr, alpha=0.6, color = 'green', linewidth = 1.0)
plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(radgyr)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot radius of gyration as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'radius_gyration_dist' #@param {type:"string"}

ax = sb.kdeplot(radgyr, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Radius of gyration ($\AA$)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute RMSF**

Selection = "Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
  axis= "Residue"
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
  axis= "Atom"
else:
  mask_end = mask_glycan
  axis= "Atom"

#@markdown **Provide output file names below:**

Output_name = 'rmsf' #@param {type:"string"}

traj_align = pt.align(traj_load, mask=mask_end, ref=0)
rmsf = pt.rmsf(traj_align, mask = mask_end)
# bfactor = pt.bfactors(traj_load, byres=False)

# Plotting:
plt.plot(rmsf[:,1], alpha=1.0, color = 'red', linewidth = 1.0)

plt.xlabel(axis, fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSF ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.xlim(0, len(rmsf[:-1]))

#plt.xticks(np.arange(min(rmsf[:1]), max(rmsf[:1])))
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsf)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **2D RMSD**

Selection = "Protein+Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
else:
  mask_end = mask_glycan

#@markdown **Provide output file names below:**

Output_name = '2D_rmsd' #@param {type:"string"}

last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

mat1 = pt.pairwise_rmsd(traj_load, mask=mask_end, frame_indices=range(int(number_frames_analysis)))


ax = plt.imshow(mat1, cmap = 'PRGn', origin='lower', interpolation = 'bicubic')
plt.title('2D RMSD')
plt.xlabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.ylabel('Time (ns)', fontsize = 14, fontweight = 'bold')
# plt.xticks(fontsize = 12)
# plt.yticks(fontsize = 12)
plt.xticks(a, b.round(decimals=3), fontsize = 12)
plt.yticks(a, b.round(decimals=3), fontsize = 12)
# plt.xlim(0, a[-1])
# plt.ylim(0, a[-1])

cbar1 = plt.colorbar()
cbar1.set_label("RMSD ($\AA$)", fontsize = 14, fontweight = 'bold')


plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat1)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Calculate eigenvectors of Principle Component Analysis (PCA)**
Selection = "Protein+Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
else:
  mask_end = mask_glycan

data = pt.pca(traj_load, fit=True, ref=0, mask=mask_end, n_vecs=2)
#print('projection values of each frame to first mode = {} \n'.format(data[0][0]))
#print('projection values of each frame to second mode = {} \n'.format(data[0][1]))
#print('eigvenvalues of first two modes', data[1][0])
#print("")
#print('eigvenvectors of first two modes: \n', data[1][1])

last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
a2 = a.tolist()
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

#@markdown **Provide output file names below:**
Output_name = 'PCA' #@param {type:"string"}

Output_PC1 = 'PC1' #@param {type:"string"}
Output_PC2 = 'PC2' #@param {type:"string"}

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
projection_data = data[0]
plt.title(r'PCA')
PC1 = data[0][0]
PC2 = data[0][1]

a = plt.scatter(PC1,PC2, c=range(int(number_frames_analysis)), cmap='Greens', marker='o',s=8, alpha=1)
plt.clim(0, last_frame)

plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
# N = len(number_frames)
# x2 = np.arange(N)

cbar1 = plt.colorbar(a, orientation="vertical")
cbar1.set_label('Time(ns)', fontsize = 14, fontweight = 'bold')
cbar1.set_ticks(a2)
cbar1.set_ticklabels(b.round(decimals=3))

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

pc1=pd.DataFrame(PC1)
pc1.to_csv(os.path.join(workDir, Output_PC1 + ".csv"))
pc2=pd.DataFrame(PC2)
pc2.to_csv(os.path.join(workDir, Output_PC2 + ".csv"))

In [ ]:
#@title **Plot Principal Component 1 (PC1) and Principal Component 2 (PC2) as a distribution**
Output_name = 'PCA_dist' #@param {type:"string"}


fig = plt.figure(figsize=(9,5))

plt.subplot(1, 2, 1)
ax = sb.kdeplot(PC1, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)
plt.subplot(1, 2, 2)
ax2 = sb.kdeplot(PC2, color="purple", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(True)
ax2.spines['left'].set_visible(False)


plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Pearson's Cross Correlation (CC)**

Selection = "Protein+Glycan" #@param ["Protein", "Protein+Glycan", "Glycan"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Glycan":
  mask_end = mask_protein_glycan
else:
  mask_end = mask_glycan

#@markdown **Provide output file names below:**

Output_name = 'cross_correlation' #@param {type:"string"}


traj_align = pt.align(traj_load, mask=mask_end, ref=0)

mat_cc = matrix.correl(traj_align, mask_end)

ax = plt.imshow(mat_cc, cmap = 'PiYG_r', interpolation = 'bicubic', vmin = -1, vmax = 1, origin='lower')

plt.xlabel('Residues', fontsize = 14, fontweight = 'bold')
plt.ylabel('Residues', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar()
cbar1.set_label('$CC_ij$', fontsize = 14, fontweight = 'bold')

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat_cc)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))